In [1]:
%matplotlib inline

import imp
import keras.backend
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import keras
from keras.datasets import mnist
from keras.models import Model
# from keras.optimizers import RMSprop
from keras import optimizers

import innvestigate
import innvestigate.applications
import innvestigate.applications.mnist
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
from innvestigate.utils.tests.networks import base as network_base
import time

from IPython.core.display import display, HTML
from innvestigate.tools import Perturbation, PerturbationAnalysis

eutils = imp.load_source("utils", "../utils.py")
mnistutils = imp.load_source("utils_mnist", "../utils_mnist.py")

Using TensorFlow backend.


# Introduction

In this experiment, we are going to build a sentiment analysis classifer, similar to [Arras et al. (2017)][arras]. In particular, we are going to predict sentiment of movie reviews. The dataset that we are going to use is [Standford Sentiment Treebank][standford]. Although the original dataset contains reviews in 5 categories, here we are interested whether reviews are positive or negative. Neutral reviews are excluded from this experiment.

![][sample]


[standford]: https://nlp.stanford.edu/sentiment/
[sample]: https://i.imgur.com/AZm1YcD.png

# Data Preparation
Code for preparing data **shall** be included later. 

We can also **ignore** this part, and simply provide a way to download prepared data as well as embedding weights

In [2]:
embeddings = np.load('../data/stanford-sentiment-treebank/embeddings.npy')
dataset_names = {
    'train': 'sequence_train.txt',
    'test': 'sequence_test_root.txt',
} 

In [3]:
embeddings_dim = embeddings.shape[1]

In [4]:
df_vocab = pd.read_csv('../data/stanford-sentiment-treebank/dict.txt', sep='::', header=None, names=['vocab'])

/Users/heytitle/projects/innvestigate/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
decode_dict = dict(zip(range(len(df_vocab)), list(df_vocab.vocab.values)))

In [6]:
df_vocab.vocab.values

array(['the', 'rock', 'is', ..., 'chevy', 'persistently', 'resuscitation'],
      dtype=object)

In [7]:
max_length = 40
num_classes= 2

In [8]:
def read_data(path):
    x = []
    y = []

    with open(path) as fp:
        for line in fp:
            tokens = np.array(line.strip().split(' ')).astype(int)

            # the vocab indices from the data start from 1
            seq = list(tokens[1:] - 1)
            x.append(seq)

            # the label in the data starts from 1-5
            y.append(tokens[0] - 1)

    return x, y


datasets = dict()
for k, v in dataset_names.items():
    x, y = read_data('../data/stanford-sentiment-treebank/%s' % v)
    y = np.array(y)
    total_samples = y.shape[0]
    xd = np.zeros((total_samples, max_length, embeddings.shape[1]))
    for i in range(total_samples):
        lx = len(x[i])
        for j, widx in enumerate(x[i]):
            if j < max_length:
                xd[i, j, :] = embeddings[widx]
            else:
                break
    indices = np.where(y != 2)
    y_selected = y[indices].reshape(-1)
    y_final = np.zeros(y_selected.shape)
    y_final[y_selected > 2] = 1
    datasets[k] = xd[indices], y_final.astype(int), np.array(x)[indices]

# Model Construction

We are going to use a convolutional neural network for this propose. The architecture was proposed in [Arras et al. (2016)][arras2]

![][arch]

[arch]: https://i.imgur.com/GE8nrWX.png
[arras2]: https://arxiv.org/abs/1612.07843

In [9]:
def build_network(input_shape, output_n, activation=None, dense_unit=256, dropout_rate=0.25):
    if activation:
        activation = "relu"

    net = {}
    net["in"] = network_base.input_layer(shape=input_shape)
    net["conv"] = keras.layers.Conv2D(filters=100, kernel_size=(1,1), strides=(1, 1), padding='valid')(net["in"])
    net["pool"] = keras.layers.MaxPooling2D(pool_size=(1, input_shape[2]), strides=(1,1))(net["conv"])
    net["out"] = network_base.dense_layer(keras.layers.Flatten()(net["pool"]), units=output_n, activation=activation)
    net["sm_out"] = network_base.softmax(net["out"])


    net.update({
        "input_shape": input_shape,

        "output_n": output_n,
    })
    return net

net = build_network((None, 1, max_length, embeddings_dim), num_classes)
model_without_softmax, model_with_softmax = Model(inputs=net['in'], outputs=net['out']), Model(inputs=net['in'], outputs=net['sm_out'])

In [10]:
data = (
   np.expand_dims(datasets['train'][0], axis=1),
   datasets['train'][1],
   np.expand_dims(datasets['test'][0], axis=1),
   datasets['test'][1]
)

In [11]:
def train_model(model, data, batch_size=128, epochs=20, num_classes=2):

    x_train, y_train, x_test, y_test = data
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(),
                  metrics=['accuracy'])

    print(x_train.shape)
    print(y_train.shape)

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1)
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [12]:
train_model(model_with_softmax, data, num_classes=2, batch_size=256, epochs=5)

(98788, 1, 40, 60)
(98788, 2)
Epoch 1/5
98788/98788 [==============================] - 10s 99us/step - loss: 0.2760 - acc: 0.8992
Epoch 2/5
98788/98788 [==============================] - 10s 103us/step - loss: 0.2047 - acc: 0.9259
Epoch 3/5
98788/98788 [==============================] - 11s 107us/step - loss: 0.1950 - acc: 0.9294
Epoch 4/5
98788/98788 [==============================] - 11s 107us/step - loss: 0.1882 - acc: 0.9330
Epoch 5/5
98788/98788 [==============================] - 8s 83us/step - loss: 0.1826 - acc: 0.9347
Test loss: 0.4600873169239113
Test accuracy: 0.8034047228762372


In [13]:
model_without_softmax.set_weights(model_with_softmax.get_weights())

# Model Analysis

In [14]:
methods = [
    # NAME                    OPT.PARAMS               POSTPROC FXN                TITLE
    ("gradient",              {},                       mnistutils.graymap,        "Gradient"),
    ("guided_backprop",       {},                       mnistutils.bk_proj,        "Guided Backprop",),
    ("lrp.z",                 {},                       mnistutils.heatmap,         "LRP-Z"),
    ("lrp.alpha_2_beta_1",    {},                       mnistutils.heatmap,         "LRP-Alpha2-Beta1"),
]

In [15]:
analyzers = [innvestigate.create_analyzer(method[0],
                                        model_without_softmax,
                                        **method[1]) for method in methods]
for analyzer in analyzers:
    analyzer.fit(data[0], pattern_type='relu', batch_size=256, verbose=1)

/Users/heytitle/projects/innvestigate/innvestigate/analyzer/base.py:113: RuntimeWarning: This analyzer does not need to be trained. Still fit() is called.
  " Still fit() is called.", RuntimeWarning)


In [16]:
label_to_class_name = {
    0 : 'negative',
    1 : 'positive'
}

In [17]:
test_sample_indices = [1718, 726, 908, 1523, 454, 539]
test_samples = []
for i in test_sample_indices:
    x = datasets['test'][0][i]

    test_samples.append( (x, datasets['test'][1][i]) )

In [18]:
analysis = np.zeros([len(test_samples), len(analyzers), 1, max_length])

In [19]:
text = []
for i, (x, y) in enumerate(test_samples):
    print('Image {}: '.format(i), end='')
    t_start = time.time()
    x = x.reshape((1, 1, max_length, embeddings_dim))
    print('')
    

    presm = model_without_softmax.predict_on_batch(x)[0] #forward pass without softmax
    prob = model_with_softmax.predict_on_batch(x)[0] #forward pass with softmax
    y_hat = prob.argmax()

    text.append(("%s" %label_to_class_name[y],    # ground truth label
                 "%.2f" %presm.max(),             # pre-softmax logits
                 "%.2f" %prob.max(),              # probabilistic softmax output  
                 "%s" %label_to_class_name[y_hat] # predicted label
                ))

    for aidx, analyzer in enumerate(analyzers):

        a = np.squeeze(analyzer.analyze(x))
        a = np.sum(a, axis=1)

        analysis[i, aidx] = a
    t_elapsed = time.time() - t_start
    print('{:.4f}s'.format(t_elapsed))

Image 0: 
1.3728s
Image 1: 
0.0040s
Image 2: 
0.0035s
Image 3: 
0.0032s
Image 4: 
0.0030s
Image 5: 
0.0035s


#  Heatmap Visualization

In [20]:
# code in this block taken from https://github.com/ArrasL/LRP_for_LSTM

def rescale_score_by_abs(score, max_score, min_score):
    """
    rescale positive score to the range [0.5, 1.0], negative score to the range [0.0, 0.5],
    using the extremal scores max_score and min_score for normalization
    """

    # CASE 1: positive AND negative scores occur --------------------
    if max_score > 0 and min_score < 0:

        if max_score >= abs(min_score):  # deepest color is positive
            if score >= 0:
                return 0.5 + 0.5 * (score / max_score)
            else:
                return 0.5 - 0.5 * (abs(score) / max_score)

        else:  # deepest color is negative
            if score >= 0:
                return 0.5 + 0.5 * (score / abs(min_score))
            else:
                return 0.5 - 0.5 * (score / min_score)

                # CASE 2: ONLY positive scores occur -----------------------------
    elif max_score > 0 and min_score >= 0:
        if max_score == min_score:
            return 1.0
        else:
            return 0.5 + 0.5 * (score / max_score)

    # CASE 3: ONLY negative scores occur -----------------------------
    elif max_score <= 0 and min_score < 0:
        if max_score == min_score:
            return 0.0
        else:
            return 0.5 - 0.5 * (score / min_score)
        
def getRGB(c_tuple):
    return "#%02x%02x%02x" % (int(c_tuple[0] * 255), int(c_tuple[1] * 255), int(c_tuple[2] * 255))

def span_word(word, normalized_score, raw_score, colormap, highlight=False, attribute="background-color"):
    return "<span style=\"{attribute}: {color}; padding: 1px;\" title=\"relevance {rel}\">{word}</span> ".format(
        attribute=attribute,
        rel=raw_score,
        color=getRGB(colormap(normalized_score)),
        word=word,
    )


def html_heatmap(method, words, scores, cmap_name="bwr", short_version=True):
    colormap = plt.get_cmap(cmap_name)

    assert len(words) == len(scores)
    max_s = max(scores)
    min_s = min(scores)

    output_text = ""

    for idx, w in enumerate(words):
        score = rescale_score_by_abs(scores[idx], max_s, min_s)
        output_text = output_text + span_word(w, score, scores[idx], colormap) + " "

    prefix = '<b>> </b>' if short_version else '<b>Heatmap(%s):</b> ' % method
    return HTML(prefix + output_text)

In [21]:
for i, idx in enumerate(test_sample_indices):

    words = [decode_dict[t] for t in list(datasets['test'][2][idx])]
    
    print('Sample %d' % idx)
    print('Sentence : %s' % ' '.join(words))
    y_true = datasets['test'][1][idx]
    y_pred = test_samples[i][1]
    print("Pred class : %s %s" % (label_to_class_name[y_true], '✓' if y_pred == y_true else '✗ (%s)' % label_to_class_name[y_true]))
                                
    
    for j, method in enumerate(methods):
        h = html_heatmap(method[0], words, analysis[i, j][0, :len(words)], short_version=False)
        display(h)

    print('')

Sample 1718
Sentence : this may not have the dramatic gut-wrenching impact of other holocaust films , but it 's a compelling story , mainly because of the way it 's told by the people who were there .
Pred class : positive ✓



Sample 726
Sentence : without heavy-handedness , dong provides perspective with his intelligent grasp of human foibles and contradictions .
Pred class : positive ✓



Sample 908
Sentence : a frantic search for laughs , with a hit-to-miss ratio that does n't exactly favour the audience .
Pred class : negative ✓



Sample 1523
Sentence : what you would end up with if you took orwell , bradbury , kafka , george lucas and the wachowski brothers and threw them into a blender .
Pred class : positive ✓



Sample 454
Sentence : a charming yet poignant tale of the irrevocable ties that bind .
Pred class : positive ✓



Sample 539
Sentence : but he loses his focus when he concentrates on any single person .
Pred class : negative ✓
